In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

import numpy as np
import pandas as pd
import time
import re 

from IPython.display import Image

#Word cloud
# from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image # 만약 "No module named 'PIL'" 에러가 발생하면 [ pip install Pillow==5.4.1 ] 로 라이브러리를 설치해줍니다.
from nltk.corpus import stopwords

하나만 테스트로 뽑아보기

In [2]:
songid = 7896508
url = "https://www.melon.com/song/detail.htm?songId="+ str(songid)

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

web = requests.get(url,headers=headers).content
web_page = BeautifulSoup(web, 'html.parser')

In [3]:
# 제목
title = web_page.find('div',{'class':'song_name'}).get_text().strip() 
title = title.split('\t')[-1] 
# 가수명
artist = web_page.find('a',{'class':'artist_name'})['title'] 
# 발매일, 장르
date_genre = web_page.find('dl',{'class':'list'}).get_text() 
date = date_genre.split('\n')[4] 
genre = date_genre.split('\n')[6] 
#가사
lyric = web_page.find('div',{'class':'lyric'}).get_text().strip() 
# get_text() 제거하면 br태그 같이 나옴
lyric_2 = web_page.find('div',{'class':'lyric'}) #.get_text()#.strip() 
# get_text() 제거하면 br태그 같이 나옴

print(title, artist, date, genre)
print()
print(lyric) 
print(type(lyric))
print()
print(lyric_2) 
print(type(lyric_2))


AttributeError: 'NoneType' object has no attribute 'get_text'

1-1) SongID 크롤링


In [ ]:
# 노래 고유 번호(song id) 가져오기 
def get_songid(chrome_path, gnrcode):
    driver = webdriver.Chrome(executable_path=chrome_path) 
    
    # 노래 고유번호
    song_id = []
    
    # url의 startindex 변경 -> Page 변경 (10페이지, 장르당 500곡)
    for start_idx in range(1, 452,50): # 1,452,50
        songlist_url = 'https://www.melon.com/genre/song_list.htm?gnrCode=' + gnrcode \
        + '&dtlGnrCode=#params%5BgnrCode%5D=' + gnrcode \
        + '&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj' \
        + '&startIndex=' + str(start_idx)

        print(songlist_url)

        driver.get(songlist_url)
        time.sleep(3)
        web = driver.page_source
        source = BeautifulSoup(web,'html.parser')

        # 노래 고유번호인 Songid를 추출
        for song_info in source.find_all('div',{'class':'ellipsis rank01'}):
            # print('크롤링 중입니다 : ', song_info.find('a')['title'][:-2])
            id = song_info.find('a')['href']
            id = re.findall("\d+", id)[-1]  
            song_id.append(id)
        print('****다음 페이지로 넘어갑니다****')

    # 하나의 장르 크롤링 완료
    print("---------------------------------------------------")
    print("장르 랩 크롤링 완료되었습니다")

    driver.close()
    driver.quit()
    
    return song_id

In [ ]:
def songid_to_info(songlist):
    #변수 담을 리스트들 생성
    titles = []
    artists = []
    dates = []
    genres = []
    lyrics = []
    lyrics_ver2 = []
    #곡 정보 받아오기
    idx=0
    for songid in songlist:
        url = "https://www.melon.com/song/detail.htm?songId="+ songid
        
        # print(url)

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web = requests.get(url,headers=headers).content
        web_page = BeautifulSoup(web, 'html.parser')
        
        # print(web_page)

        #노래 제목 줄바꿈 제거
        title = web_page.find('div',{'class':'song_name'}).get_text().strip().split('\t')[-1]
        if title:
            #title = title.split('(')[0].strip() # (feat.) 등 제거
            titles.append(title)
        else:
            titles.append(' ')
        
        #가수명
        artist = web_page.find('a',{'class':'artist_name'})['title']
        if artist:
            artists.append(artist)
        else:
            artists.append(' ')
            
        #곡 발매일, 장르
        date_genre = web_page.find('dl',{'class':'list'}).get_text()
        if date_genre:
            date = date_genre.split('\n')[4]
            genre = date_genre.split('\n')[6]
            dates.append(date)
            genres.append(genre)
        else:
            dates.append(' ')
            genres.append(' ')

        #가사
        try: 
            lyric = web_page.find('div',{'class':'lyric'}).get_text().strip()
            if lyrics:
                lyrics.append(lyric)
            else:
                lyrics.append(' ')
        except: # 성인인증이 필요한 경우 스크래핑 불가 -> 가사 생략
            print('----{} : 성인인증이 필요한 가사입니다----'.format(title))
            lyrics.append(' ')

        #가사 ver2
        try: 
            lyric_2 = web_page.find('div',{'class':'lyric'}) #.get_text().strip()
            if lyrics_ver2:
                lyrics_ver2.append(lyric_2)
            else:
                lyrics_ver2.append(' ')
        except: # 성인인증이 필요한 경우 스크래핑 불가 -> 가사 생략
            print('----{} : 성인인증이 필요한 가사입니다----'.format(title))
            lyrics_ver2.append(' ')
             

        idx += 1 #다음곡으로 넘어가기
        if idx%100 ==0:
            print(idx, '번째곡 크롤링 완료 :', title)

    info_dict = {'Title':titles, 
                 'Artist':artists, 
                 'Date':dates, 
                 'Genre':genres, 
                 'Lyric':lyrics,
                 'Lyric_ver2': lyrics_ver2}
    print(len(titles), len(artists), len(dates), len(genres), len(lyrics), len(lyrics_ver2))
    df_song_info = pd.DataFrame(info_dict)
    return df_song_info

In [ ]:
# 개인 크롬 path 삽입 
chrome_path = '/opt/homebrew/bin/chromedriver'

# 맡은 장르 적어주세요 
genre_name = '댄스'

In [ ]:
# 장르와 장르 id 매칭 
gnr_info = {'발라드':'GN0100', '댄스':'GN0200', '랩/힙합':'GN0300', 'R&B/Soul':'GN0400',
           '인디음악':'GN0500', '록/메탈':'GN0600', '트로트':'GN0700','포크/블루스':'GN0800'}

# song_id 가져오기 
song_id = get_songid(chrome_path, gnr_info[genre_name])

# 노래 정보 가져오기 
df_genre_songid_to_info = songid_to_info(song_id)

# 엑셀 반환 
filename = "{0}_500songs.xlsx".format(genre_name) 
df_genre_songid_to_info.to_excel(filename, index=False, encoding='utf-8')

/var/folders/rz/3h16f8bd5s36k1xv36vgkl1m0000gn/T/ipykernel_10179/399682017.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path)


https://www.melon.com/genre/song_list.htm?gnrCode=GN0200&dtlGnrCode=#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0200&dtlGnrCode=#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=51
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0200&dtlGnrCode=#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=101
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0200&dtlGnrCode=#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=151
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0200&dtlGnrCode=#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP

AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
df_genre_songid_to_info

NameError: name 'df_genre_songid_to_info' is not defined

In [ ]:
df_genre_songid_to_info.Lyric.iloc[1]

'우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록네게 말할 거야너를 좋아한다고안녕 그대야요즘 어떻게 지내가로수길 걷다가생각이 나서행복해 보이는 사람들 속에우리 둘만 없어서어색했었어늘 집에 갈 땐항상 뒤돌아보면날 보며 웃어주던네가 생각났어우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록네게 말할 거야너를 좋아한다고토요일마다 가던한강공원에이어폰 하나씩나눠끼고서벚꽃엔딩같이 듣고 있으면부러울 게 없었어너만 있으면널 사랑할 때내가 너무 그리워날 보며 웃어주던네가 보고 싶어우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록네게 말할 거야너를 좋아한다고같이 별 보러 갈래널 다시 만나면네 옆에 기대서잠들고 싶어너의 품에 안길래이 밤이 새도록내 곁에 있어줘내겐 너뿐이라고'

In [ ]:
df_genre_songid_to_info.Lyric_ver2.iloc[1]

<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->
							우리 다시 만날래<br/>예쁘게 빛나던<br/>모든 추억들이<br/>너무 그리워<br/>너의 품에 안길래<br/>이 밤이 새도록<br/>네게 말할 거야<br/>너를 좋아한다고<br/>안녕 그대야<br/>요즘 어떻게 지내<br/>가로수길 걷다가<br/>생각이 나서<br/>행복해 보이는 사람들 속에<br/>우리 둘만 없어서<br/>어색했었어<br/>늘 집에 갈 땐<br/>항상 뒤돌아보면<br/>날 보며 웃어주던<br/>네가 생각났어<br/>우리 다시 만날래<br/>예쁘게 빛나던<br/>모든 추억들이<br/>너무 그리워<br/>너의 품에 안길래<br/>이 밤이 새도록<br/>네게 말할 거야<br/>너를 좋아한다고<br/>토요일마다 가던<br/>한강공원에<br/>이어폰 하나씩<br/>나눠끼고서<br/>벚꽃엔딩같이 듣고 있으면<br/>부러울 게 없었어<br/>너만 있으면<br/>널 사랑할 때<br/>내가 너무 그리워<br/>날 보며 웃어주던<br/>네가 보고 싶어<br/>우리 다시 만날래<br/>예쁘게 빛나던<br/>모든 추억들이<br/>너무 그리워<br/>너의 품에 안길래<br/>이 밤이 새도록<br/>네게 말할 거야<br/>너를 좋아한다고<br/>같이 별 보러 갈래<br/>널 다시 만나면<br/>네 옆에 기대서<br/>잠들고 싶어<br/>너의 품에 안길래<br/>이 밤이 새도록<br/>내 곁에 있어줘<br/>내겐 너뿐이라고<br/>
</div>

In [ ]:
test = "      안녕하세요 \t\t      그런데요\t\t 아니요       "
test2 = "\t\t dkdldljdah \t\t"

In [ ]:
test.strip()

'안녕하세요 \t\t      그런데요\t\t 아니요'

In [ ]:
test.strip().split('\t')

['안녕하세요 ', '', '      그런데요', '', ' 아니요']

In [ ]:
test.strip().split('\t')[-1]

' 아니요'